# MCP ToolSpec

This tool connects to MCP Servers and allows an Agent to call the tools provided by MCP Servers.

This idea is migrated from [Integrate MCP tools into LlamaIndex](https://psiace.me/posts/integrate-mcp-tools-into-llamaindex/).

To run this example, you need to edit the .env file to set the correct values for the IPInfo API Token and Azure OpenAI API.

```bash
cp .env.example .env
```

Then, start the MCP server.

```bash
python examples/mcp_server.py --server_type=sse
```


In [ ]:
from llama_index.llms.azure_openai import AzureOpenAI
import os
import dotenv

dotenv.load_dotenv()

llm = AzureOpenAI(
    model=os.environ.get("AZURE_OPENAI_MODEL", "your-model-name"),
    max_tokens=int(os.environ.get("AZURE_OPENAI_MAX_TOKENS", 4096)),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "your-azure-endpoint"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", "your-api-key"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "your-api-version"),
    engine=os.environ.get("AZURE_OPENAI_ENGINE", "your-deployment-name"),
)

In [ ]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.llms import ChatMessage
from llama_index.core.agent import ReActAgent, ReActChatFormatter
from llama_index.core.agent.react.prompts import REACT_CHAT_SYSTEM_HEADER

SYSTEM_PROMPT = """\
You are an AI assistant.

Before you answer the user's question, you need to fetch the ip info first, to help you follow the laws of the country.

- When user ask about current ip info, location, city, country or timezone, use the 'fetch_ipinfo' tool to provide the answer.
"""


async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = ReActAgent.from_tools(
        llm=llm,
        tools=list(tools),
        react_chat_formatter=ReActChatFormatter(
            system_header=SYSTEM_PROMPT + "\n" + REACT_CHAT_SYSTEM_HEADER,
        ),
        max_iterations=20,
        verbose=True,
    )
    return agent


async def handle_user_message(message_content: str, agent: ReActAgent):
    user_message = ChatMessage.from_str(role="user", content=message_content)
    response = await agent.achat(message=user_message.content)
    print(response.response)

In [ ]:
import asyncio
import nest_asyncio
from llama_index.tools.mcp.client import BasicMCPClient

nest_asyncio.apply()


async def main():
    # We consider there is a mcp server running on 127.0.0.1:8000, or you can use the mcp client to connect to your own mcp server.
    mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
    mcp_tool = McpToolSpec(client=mcp_client)

    agent = await get_agent(mcp_tool)
    await handle_user_message("what is the current ip info?", agent)
    await handle_user_message("Is it legal to hold drugs in my country?", agent)


asyncio.run(main())

> Running step 7eb32583-ed74-43f6-aff9-9fe64a151500. Step input: what is the current ip info?


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}